In [11]:
import os
import datetime
import logging
import sys
import time
import getpass
import pandas 
import requests

from textwrap import dedent

import toloka.client as toloka
import toloka.client.project.template_builder as tb

# False - if problems with tasks in toloka-kit still occure
USE_KIT_FOR_TASKS = True 

PASSWORD = os.environ.get(
    "TOLOKA_PWD", 
    getpass.getpass('Enter your OAuth token: ')
)

# PRODUCTION or SANDBOX, SANDBOX is default
ENVIRONMENT = os.environ.get("TOLOKA_ENV", "SANDBOX")

N_TASKS = os.environ.get("N_TASKS", 3)

logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)


Enter your OAuth token:  ········


In [2]:
toloka_client = toloka.TolokaClient(
    PASSWORD, ENVIRONMENT
)
print(toloka_client.get_requester())


[DEBUG] httpx: load_ssl_context verify=True cert=None trust_env=True http2=False
[DEBUG] httpx: load_verify_locations cafile='/home/fatuus/deepschool-cvr-toloka/venv/lib/python3.10/site-packages/certifi/cacert.pem'
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7feb469d8460>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7feb7dd397c0> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7feb469d8430>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'GET']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'GET']>
[DEBUG] httpcore.http11: send_request_

### Preparing Web UI

In [3]:
project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ImageAnnotationFieldV1(  # Component that selects areas in images
        tb.OutputData('result'),  # Path for writing output data
        tb.InputData('image'),  # Getter for the input image
        shapes={'rectangle': True},  # Allow to select only rectangular areas
        validation=tb.RequiredConditionV1(hint='Please select an area')  # At least one area should be selected
    )
)

# You can write instructions and upload them from a file or enter them later in the web interface
# prepared_instruction = open('instruction.html').read().strip()
prepared_instruction = '<b>Outline each traffic sign with a separate bounding box(rectangle).</b>'

# Set up the project
detection_project = toloka.Project(
    public_name='Outline all traffic signs with bounding boxes(toloka-kit)',
    public_description='Find and outline all traffic signs with bounding boxes.',
    public_instructions=prepared_instruction,
    # Set up the task: view, input, and output parameters
    task_spec=toloka.project.task_spec.TaskSpec(
        input_spec={'image': toloka.project.UrlSpec()},
        output_spec={'result': toloka.project.JsonSpec()},
        view_spec=project_interface,
    ),
)

In [4]:
detection_project = toloka_client.create_project(detection_project)


[DEBUG] httpcore.connection: close.started
[DEBUG] httpcore.connection: close.complete
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7feb4697d000>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7feb7dd397c0> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7feb7c41ab90>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_body.complete
[DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: r

### Creating a pool

In [7]:
# Create a pool
new_pool = toloka.Pool(
    project_id=detection_project.id,
    private_name='Pool 1',
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    reward_per_assignment=0.01, # USD
    auto_accept_solutions=False,
    auto_accept_period_day=10,
    assignment_max_duration_seconds=60*10,
    defaults=toloka.Pool.Defaults(default_overlap_for_new_task_suites=1)
)

new_pool = toloka_client.create_pool(new_pool)


[DEBUG] httpcore.connection: close.started
[DEBUG] httpcore.connection: close.complete
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7feb4665c8e0>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7feb7dd397c0> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7feb4665ef50>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_body.complete
[DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: r

In [12]:
tasks = [
    toloka.Task(
        input_values={
            "image": url
        }, 
        pool_id=new_pool.id
    ) for url in urls
]
toloka_client.create_tasks(tasks, allow_defaults=True)
print(f'Populated pool with {len(tasks)} tasks')
# print(f'To view this pool, go to https://sandbox.toloka.yandex.com/requester/project/{new_project.id}/pool/{new_pool.id}')


[DEBUG] httpcore.connection: close.started
[DEBUG] httpcore.connection: close.complete
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7feb4672a650>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7feb7dd397c0> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7feb4672add0>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_body.complete
[DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: r

  0%|                                                   | 0/100 [00:00<?, ?it/s]

[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'GET']>


  0%|                                                   | 0/100 [00:00<?, ?it/s]

[DEBUG] httpcore.http11: send_request_headers.complete


...

[DEBUG] httpcore.http11: response_closed.complete


100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 33.51it/s]

[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'GET']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'GET']>
[DEBUG] httpcore.http11: send_request_body.complete
[DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'GET']>


[DEBUG] httpcore.http11: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 31 May 2023 19:02:53 GMT'), (b'Content-Type', b'application/json;charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'x-tlk-request-id', b'5b02cd09b6fb96e17303e698ad42b8ac'), (b'x-api-ver', b'16.release-278.29044.70d6c91'), (b'vary', b'Origin'), (b'vary', b'Access-Control-Request-Method'), (b'vary', b'Access-Control-Request-Headers'), (b'strict-transport-security', b'max-age=31536000'), (b'x-content-type-options', b'nosniff'), (b'x-xss-protection', b'1; mode=block'), (b'cache-control', b'no-cache, no-store, must-revalidate, max-age=0'), (b'x-frame-options', b'DENY'), (b'x-content-type-options', b'nosniff'), (b'x-qe-streaming', b'true'), (b'vary', b'Accept-Encoding, User-Agent'), (b'content-encoding', b'gzip')])
[INFO] httpx: HTTP Request: GET https://sandbox.toloka.dev/api/v1/operations/57562e3a-c609-4567-928b-a5b208696c52/log "HTTP/1.1 

NameError: name 'new_project' is not defined

In [10]:
urls = [
    "https://2.downloader.disk.yandex.ru/preview/4d586b37bdaa2c56feac0df50988ace5692b7d8171de02ef11746e7c7027eb61/inf/HXXK3JGF4-pHH7QlKGowRbiNAs53ek8hEPIjru4DeDDzU30X0XYLZQ6HrBCzGau1BtHXYs1LiBsYy3pBf0vuaQ%3D%3D?uid=135551846&filename=barcode_03.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=135551846&tknv=v2&size=2066x1368",
    "https://2.downloader.disk.yandex.ru/preview/16ef8969eeb9d96fb2260a58736a7e03bd1f5e4ff663c3a40ed3e3c16dc943c9/inf/RU0Sq2RN6Cb5MUd01dMhZLiNAs53ek8hEPIjru4DeDAzNByppXSizKVS6H2CCfRSyJeW_0q9BaI4bD6PVOd88g%3D%3D?uid=135551846&filename=barcode_04.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=135551846&tknv=v2&size=2066x1368"
]

if USE_KIT_FOR_TASKS:
    tasks = [
        toloka.Task(
            input_values={
                "image": url
            }, 
            pool_id=new_pool.id
        ) for url in urls
    ]
    toloka_client.create_tasks(tasks, allow_defaults=True)
    print(f'Populated pool with {len(tasks)} tasks')
    print(f'To view this pool, go to https://sandbox.toloka.yandex.com/requester/project/{new_project.id}/pool/{new_pool.id}')
else:
    if ENVIRONMENT == "PRODUCTION":
        url = "https://toloka.dev/api/v1/tasks"
    else:
        url = "https://sandbox.toloka.dev/api/v1/tasks"
    headers = {
      'Authorization': f"OAuth {PASSWORD}",
      'Content-Type': 'application/json'
    }
    for url in urls:
        payload = {
          "input_values": {
            "image": url
          },
          "pool_id": f"{new_pool.id}"
        }
        response = requests.post(url, headers=headers, json=payload)
        print(response.text)

[DEBUG] urllib3.connectionpool: Starting new HTTPS connection (1): 2.downloader.disk.yandex.ru:443
[DEBUG] urllib3.connectionpool: https://2.downloader.disk.yandex.ru:443 "POST /preview/4d586b37bdaa2c56feac0df50988ace5692b7d8171de02ef11746e7c7027eb61/inf/HXXK3JGF4-pHH7QlKGowRbiNAs53ek8hEPIjru4DeDDzU30X0XYLZQ6HrBCzGau1BtHXYs1LiBsYy3pBf0vuaQ%3D%3D?uid=135551846&filename=barcode_03.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=135551846&tknv=v2&size=2066x1368 HTTP/1.1" 200 8792
[DEBUG] charset_normalizer: Encoding detection: Unable to determine any suitable charset.
�a��5!�G�g{��ܕ��"� d Jm�lX�co��� \�FG��ۺذ̼�1�zn��t����"e��M��w�`��4��C��<'�H��pfσ��".[�0�=�><��6��a��5!�G�r�Gǔ�����L>2�;��G���M�r��t��նn���s�d\��?i�m��83��M�r��m��rW��Gx>i��Y�3���"�J��� 񿦅�ap�5!�G�f>� ��4,������<@l�8rA�Rl�o������ftp��>٘,����� Α�sT�[5�77`��+�7H�G��B-�����Eܕ� l�t��նn�	7a�~M�����?i�m��5�M�a�e��#{�Eܕ��xG�=4l��݂.�H����?!����o;	��l�2��&